In [2]:
import numpy as np
from datetime import datetime as dt
import pandas as pd, datetime as dt

In [3]:
cd C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data

C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data


In [4]:
user = pd.read_csv('user_processed.csv',sep=',')
listening = pd.read_csv('listening_processed.csv',sep=',')
order = pd.read_csv('order_processed.csv',sep=',')
clustering = pd.read_csv('clustering_processed.csv',sep=',')


In [5]:
# change column name
order = order.rename(columns = {"OrderID (PK)": "oderID", "Transaction Date" : "transaction_date","UserID (FK)" : "userID", "Bill Type": "bill_type", "Total Amount": "total_amount" })
clustering = clustering.rename(columns = {"User_ID (FK)": "userID", "Type_user": "type_user", "MainCluster_ID" : "main_cluseringID", "MainCluster_Description" : "main_clustering_description"})
listening = listening.rename(columns = {"PlaylistID (PK)": "playlistID", "Playlist Name":"playlist_name", "Category":"cat", "Sub Category" :"sub_cat", "Playlist Type" :"palylist_type","Playlist Duration (min)": "playlist_duration","Actual Duration (min)":"actual_duation","UserID (FK)":"userID", "Listening Time":"listening_time", "Listening Date" :"listening_date", "Listening Datetime":"listening_datetime"})
user = user.rename(columns = {"UserID (PK)":"userID", "VIP hay Free":"user_type", "Gender":"gender", "Year of Birth":"year_of_birth","Age Range": "age_range", "Age":"age", "Region": "region","Operation Systems":"operation_system","Registration Date":"registration_date", "Total Listerning":"total_listening", "Total Listening Time (Min)":"total_listening_time",   })

# change date time
# xu li datetime
order.transaction_date = pd.to_datetime(order['transaction_date'], format='%Y-%m-%d')
user.registration_date = pd.to_datetime(user['registration_date'], format='%Y-%m-%d')
listening.listening_datetime = pd.to_datetime(listening['listening_datetime'], format='%Y-%m-%d')

# change date by weekly
def adding_by_week(df, date_field):
    # df["by_week"] = df[[date_field]].copy()
    # df.resample('W-Mon', on=date_field).sum().reset_index().sort_values(by=date_field)
    # order1.resample('W-Mon', on='transaction_date').sum().reset_index().sort_values(by='transaction_date')
    df['by_week'] = df.apply(lambda row: row[date_field] - dt.timedelta(days=row[date_field].weekday()), axis=1)
    df = df['by_week'].groupby(df['by_week']).count()

adding_by_week(order, "transaction_date")
adding_by_week(user, "registration_date")
adding_by_week(listening, "listening_datetime")

In [6]:
## fillna for the user Table

user.operation_system = user.operation_system.fillna('none')
user["gender"] = user.gender.replace("no information","none")
user["gender"] = user.gender.replace("other","none")

## add some features related to time of the listening Table

listening['hour_listening'] = listening.listening_datetime.dt.hour
listening['dayofweek'] = listening['listening_datetime'].dt.dayofweek

listening["timepoint_of_the_day"] = listening.hour_listening.apply(lambda x : 
                                                             "late night" if x < 4 else
                                                             "early morning" if x < 7 else
                                                             "morning" if x < 11 else
                                                             "early afternoon" if x < 16 else
                                                             "afternoon" if x < 18 else
                                                             "evening"
                                                            )

listening['dayofweek'] =listening.dayofweek.apply(lambda x :  "Mon" if x == 0 else
                                                             "Tue" if x ==1 else
                                                             "Wed" if x ==2 else
                                                             "Thu" if x == 3 else
                                                             "Fri" if x == 4 else
                                                             "Sat" if x == 5 else
                                                             "Sun" 
                                                            )

listening['Workingday'] = listening["dayofweek"].apply(lambda x : 0 if x == 'Sun' or x == 'Sat' else 1)


In [7]:
user.head(3)

,userID,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week
0,86229,free,male,NaN,No information,NaN,VN,android,2020-11-27,218,4151.3,2020-11-23
1,48053,free,male,1995.0,25 - 34,26.0,VN,ios,2020-08-24,19,278.6,2020-08-24
2,46340,free,male,1988.0,25 - 34,33.0,VN,website,2020-08-19,28,170.5,2020-08-17


In [8]:
listening.head(3)

,playlistID,playlist_name,cat,sub_cat,palylist_type,playlist_duration,actual_duation,userID,listening_time,listening_date,listening_datetime,Unnamed: 11,by_week,hour_listening,dayofweek,timepoint_of_the_day,Workingday
0,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,0.1,82846,06:05:00,03-12-2020,2020-12-03 06:05:00,NaN,2020-11-30 06:05:00,6,Thu,early morning,1
1,4577,Tư Duy Tích Cực - Bạn Là Những Gì Bạn Nghĩ (Nh...,Sách nói,Hạnh phúc,vip,79.8,15.9,56112,12:49:00,30-10-2020,2020-10-30 12:49:00,NaN,2020-10-26 12:49:00,12,Fri,early afternoon,1
2,4169,50 thiên thần của bạn: Giải phóng bản thân khỏ...,Sách nói,Tâm linh,vip,324.8,6.8,94362,00:17:00,15-12-2020,2020-12-15 00:17:00,NaN,2020-12-14 00:17:00,0,Tue,late night,1


### mục tiêu là mình làm xem nhóm nào đóng góp nhiều revenue nhất
1. paid user: dựa vào total amount
2. free user: dựa vào duration, số lượt nghe

In [9]:
order.head()

,oderID,transaction_date,userID,bill_type,total_amount,by_week
0,6706,2020-08-16,44822,VIP,199000,2020-08-10
1,6998,2020-08-23,47128,VIP,199000,2020-08-17
2,6927,2020-08-21,47134,VIP,199000,2020-08-17
3,7054,2020-08-25,45972,VIP,199000,2020-08-24
4,6929,2020-08-21,43556,VIP,199000,2020-08-17


In [10]:
total_amount = order.groupby('userID').sum()
sum_order = total_amount.total_amount.sum()
total_amount.head()

,oderID,total_amount
userID,,
40843,2544,27000
40861,7707,199000
41003,9599,599000
41014,15326,398000
41057,9313,599000


In [11]:
user_money = pd.merge(user,total_amount, how='left',left_on='userID', right_on='userID')
total = pd.merge(clustering,user_money, how='left',left_on='userID', right_on='userID')
total.head()

,userID,type_user,main_cluseringID,main_clustering_description,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week,oderID,total_amount
0,40834,free,1,Hibernating,free,none,NaN,No information,NaN,QT,ios,2020-07-01,2,8.7,2020-06-29,NaN,NaN
1,40854,free,1,Hibernating,free,female,1999.0,18 - 24,22.0,VN,website,2020-07-01,444,20988.0,2020-06-29,NaN,NaN
2,40876,free,1,Hibernating,free,none,NaN,No information,NaN,VN,ios,2020-07-01,3,0.8,2020-06-29,NaN,NaN
3,40883,free,1,Hibernating,free,none,NaN,No information,NaN,VN,ios,2020-07-01,5,2.9,2020-06-29,NaN,NaN
4,40891,free,1,Hibernating,free,none,NaN,No information,NaN,VN,ios,2020-07-01,13,71.5,2020-06-29,NaN,NaN


### Paid contribution
1. by cluserter

In [12]:
df_paid_cluster = pd.DataFrame(total[total['type_user']=='paid'].groupby('main_clustering_description').total_amount.sum())
df_paid_cluster['%']=df_paid_cluster.total_amount/sum_order*100
df_paid_cluster.sort_values('%',ascending=False)

,total_amount,%
main_clustering_description,,
Loyal Users,262471000.0,36.638056
Potential Users,145295000.0,20.281579
New Paid Users,119967000.0,16.746070
Lost High-value,70042000.0,9.777090
Churn App Users,65260000.0,9.109576
Lost Users,50019000.0,6.982101
Need attention,3335000.0,0.465529


In [15]:
new = df_paid_cluster.sort_values('%',ascending=False)
new.to_csv('contribution_paid_cluster.csv',index=True)

In [16]:
pd.read_csv('contribution_paid_cluster.csv')

,main_clustering_description,total_amount,%
0,Loyal Users,262471000.0,36.638056
1,Potential Users,145295000.0,20.281579
2,New Paid Users,119967000.0,16.746070
3,Lost High-value,70042000.0,9.777090
4,Churn App Users,65260000.0,9.109576
5,Lost Users,50019000.0,6.982101
6,Need attention,3335000.0,0.465529


In [217]:
total_paid = total[total['type_user']=='paid']

In [222]:
def cal_contribution(df,col):
    sum_table = pd.DataFrame(df.groupby(col).total_amount.sum())
    sum_table['%']=sum_table.total_amount/sum_order*100
    return sum_table.sort_values('%',ascending=False)

In [223]:
pgender_con = cal_contribution(total_paid,'gender')
pgender_con

,total_amount,%
gender,,
male,293082000.0,40.911013
none,241612000.0,33.726369
female,181695000.0,25.362617


In [224]:
pagerange_con = cal_contribution(total_paid,'age_range')
pagerange_con

,total_amount,%
age_range,,
No information,366966000.0,51.224405
25 - 34,132483000.0,18.493165
35 - 44,100608000.0,14.043767
18 - 24,79221000.0,11.058378
45 - 54,23205000.0,3.239162
55 - 64,8711000.0,1.215959
Over 65,3741000.0,0.522202
Below 18,1454000.0,0.202962


In [225]:
preagion_con = cal_contribution(total_paid,'region')
preagion_con

,total_amount,%
region,,
VN,642795000.0,89.72709
QT,73594000.0,10.27291


In [226]:
pos_con = cal_contribution(total_paid,'operation_system')
pos_con

,total_amount,%
operation_system,,
ios,440526000.0,61.492569
android,198966000.0,27.773458
website,70357000.0,9.821061
none,6540000.0,0.912912


### Contribution of free user
Dựa trên số lượng nghe và số lượt nghe

In [239]:
total_free = total[total['type_user']=='free']
tong_lan_nghe = total_free['total_listening'].sum()
tong_tg_nghe = total_free['total_listening_time'].sum()

print('Tổng lượng nghe',  tong_tg_nghe)
total_free.head()

Tổng lượng nghe 13482242.999999998


,userID,type_user,main_cluseringID,main_clustering_description,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week,oderID,total_amount
0,40834,free,1,Hibernating,free,none,NaN,No information,NaN,QT,ios,2020-07-01,2,8.7,2020-06-29,NaN,NaN
1,40854,free,1,Hibernating,free,female,1999.0,18 - 24,22.0,VN,website,2020-07-01,444,20988.0,2020-06-29,NaN,NaN
2,40876,free,1,Hibernating,free,none,NaN,No information,NaN,VN,ios,2020-07-01,3,0.8,2020-06-29,NaN,NaN
3,40883,free,1,Hibernating,free,none,NaN,No information,NaN,VN,ios,2020-07-01,5,2.9,2020-06-29,NaN,NaN
4,40891,free,1,Hibernating,free,none,NaN,No information,NaN,VN,ios,2020-07-01,13,71.5,2020-06-29,NaN,NaN


In [243]:
def cal_contribution_free(df,col):
    df = df.groupby(col)['total_listening','total_listening_time'].sum()
    df['%tg'] = df['total_listening_time']/tong_tg_nghe*100
    df['%lan'] = df['total_listening']/tong_lan_nghe*100
    df['do_dai_1_lan'] = df['total_listening_time']/df['total_listening']
    return df.sort_values('%tg',ascending = False)

In [246]:
fcluster_con = cal_contribution_free(total_free,'main_clustering_description')
fcluster_con

<ipython-input-243-b87667b8e41c>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(col)['total_listening','total_listening_time'].sum()


,total_listening,total_listening_time,%tg,%lan,do_dai_1_lan
main_clustering_description,,,,,
"New Users, Engaging Listeners",293716,5013363.4,37.184936,31.969193,17.068745
"Potential, Engaging Listeners",199520,3345557.8,24.814549,21.716533,16.768032
Need Attention,93790,1229362.2,9.118380,10.208469,13.107604
"Loyal Users, Engaging Listeners",55078,929509.0,6.894320,5.994904,16.876230
Lost active user,62067,812112.9,6.023574,6.755614,13.084456
"New User, Skimming Listeners",78638,767063.2,5.689433,8.559266,9.754358
Hibernating,37190,505305.0,3.747930,4.047904,13.587120
Defrost,33795,462726.0,3.432114,3.678379,13.692144
"Potential, Skimming Listeners",36747,268421.5,1.990926,3.999687,7.304583


In [247]:
fos_con = cal_contribution_free(total_free,'operation_system')
fos_con

<ipython-input-243-b87667b8e41c>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(col)['total_listening','total_listening_time'].sum()


,total_listening,total_listening_time,%tg,%lan,do_dai_1_lan
operation_system,,,,,
android,520953,7010722.5,51.999675,56.702552,13.457495
ios,325496,5224174.5,38.748556,35.428252,16.049888
website,59453,1057830.2,7.846099,6.471096,17.792714
none,12845,189515.8,1.405670,1.398100,14.754052


In [248]:
fregion_con = cal_contribution_free(total_free,'region')
fregion_con

<ipython-input-243-b87667b8e41c>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(col)['total_listening','total_listening_time'].sum()


,total_listening,total_listening_time,%tg,%lan,do_dai_1_lan
region,,,,,
VN,873781,12842204.8,95.252732,95.105726,14.697281
QT,44966,640038.2,4.747268,4.894274,14.233826


In [249]:
fage_con = cal_contribution_free(total_free,'age_range')
fage_con

<ipython-input-243-b87667b8e41c>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(col)['total_listening','total_listening_time'].sum()


,total_listening,total_listening_time,%tg,%lan,do_dai_1_lan
age_range,,,,,
No information,653934,9002173.0,66.770589,71.176722,13.766180
25 - 34,105562,1780872.8,13.209025,11.489779,16.870397
18 - 24,84141,1272508.5,9.438404,9.158234,15.123525
35 - 44,59714,1138381.9,8.443565,6.499504,19.063903
45 - 54,8803,208536.1,1.546746,0.958153,23.689208
Below 18,4446,44312.4,0.328672,0.483920,9.966802
55 - 64,1872,29958.7,0.222209,0.203756,16.003579
Over 65,275,5499.6,0.040791,0.029932,19.998545


In [250]:
fgender_con = cal_contribution_free(total_free,'gender')
fgender_con

<ipython-input-243-b87667b8e41c>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(col)['total_listening','total_listening_time'].sum()


,total_listening,total_listening_time,%tg,%lan,do_dai_1_lan
gender,,,,,
male,397255,6253069.8,46.380041,43.238781,15.740695
none,280693,3724393.8,27.624438,30.551719,13.268567
female,240799,3504779.4,25.995522,26.209501,14.554792


### High values user:
1. Paid: chi trả > 199.000
2. Free: nghe tb tuần >=3

=> chọn từ clustering của nhóm
